In [2]:
import numpy as np
from sklearn.metrics import r2_score
import torch
import os, gc
from climsim_utils.data_utils import *
from tqdm import tqdm

2025-02-18 19:13:42.700218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 19:13:42.860481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-18 19:13:42.901370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 19:13:43.134495: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:

input_path = '/pscratch/sd/j/jerrylin/hugging/E3SM-MMF_ne4/preprocessing/v2_rh_mc_full/scoring_set/scoring_input.npy'
target_path = '/pscratch/sd/j/jerrylin/hugging/E3SM-MMF_ne4/preprocessing/v2_rh_mc_full/scoring_set/scoring_target.npy'
preds_path = '/pscratch/sd/j/jerrylin/hugging/E3SM-MMF_ne4/preprocessing/v2_rh_mc_full/scoring_set/scoring_set/'

grid_path = '/global/cfs/cdirs/m4334/jerry/climsim3_dev/grid_info/ClimSim_low-res_grid-info.nc'
norm_path = '/global/cfs/cdirs/m4334/jerry/climsim3_dev/preprocessing/normalizations/'

unet_model_path = '/pscratch/sd/k/kfrields/hugging/E3SM-MMF_saved_models/unet_adam'

model_paths = {'unet': unet_model_path}

model_colors = {'unet': 'blue'}

model_labels = {'unet': 'pure resLSTM'}

num_models = len(model_paths)
model_preds = {}
r2_scores = {}
r2_scores_capped = {}
zonal_heating_r2 = {}
zonal_moistening_r2 = {}

input_mean_file = 'inputs/input_mean_v6_pervar.nc'
input_max_file = 'inputs/input_max_v6_pervar.nc'
input_min_file = 'inputs/input_min_v6_pervar.nc'
output_scale_file = 'outputs/output_scale_std_lowerthred_v6.nc'
lbd_qn_file = 'inputs/qn_exp_lambda_large.txt'



grid_info = xr.open_dataset(grid_path)
input_mean = xr.open_dataset(norm_path + input_mean_file)
input_max = xr.open_dataset(norm_path + input_max_file)
input_min = xr.open_dataset(norm_path + input_min_file)
output_scale = xr.open_dataset(norm_path + output_scale_file)
lbd_qn = np.loadtxt(norm_path + lbd_qn_file, delimiter = ',')





In [6]:
data = data_utils(grid_info = grid_info, 
                  input_mean = input_mean, 
                  input_max = input_max, 
                  input_min = input_min, 
                  output_scale = output_scale,
                  qinput_log = False)
data.set_to_v2_rh_mc_vars()

input_sub, input_div, out_scale = data.save_norm(write=False) # this extracts only the relevant variables
input_sub = input_sub[None, :]
input_div = input_div[None, :]
out_scale = out_scale[None, :]

lat = grid_info['lat'].values
lon = grid_info['lon'].values
lat_bin_mids = data.lat_bin_mids

In [14]:
npy_input = np.load(input_path)

In [17]:
data.ps_index


360

In [25]:
npy_input.shape

(1681920, 557)

In [33]:
data.hyam

array([7.83478113e-05, 1.41108318e-04, 2.52923297e-04, 4.49250635e-04,
       7.86346161e-04, 1.34735576e-03, 2.24477729e-03, 3.61643148e-03,
       5.61583643e-03, 8.40325322e-03, 1.21444894e-02, 1.70168280e-02,
       2.32107981e-02, 3.09143463e-02, 4.02775807e-02, 5.13746323e-02,
       6.41892284e-02, 7.86396576e-02, 9.46300920e-02, 1.12091274e-01,
       1.30977804e-01, 1.51221318e-01, 1.71834677e-01, 1.79455096e-01,
       1.74312038e-01, 1.69085103e-01, 1.63810524e-01, 1.58490091e-01,
       1.53096986e-01, 1.47591258e-01, 1.41935802e-01, 1.36105697e-01,
       1.30089487e-01, 1.23886244e-01, 1.17504126e-01, 1.10963956e-01,
       1.04306593e-01, 9.75986738e-02, 9.09302689e-02, 8.44015017e-02,
       7.81015945e-02, 7.20893971e-02, 6.63850677e-02, 6.09771059e-02,
       5.58405513e-02, 5.09562608e-02, 4.63213463e-02, 4.19465312e-02,
       3.78435917e-02, 3.40108582e-02, 3.04246539e-02, 2.70402809e-02,
       2.38007872e-02, 2.06484555e-02, 1.75340274e-02, 1.44211300e-02,
      

In [36]:
lbd_qn


array([10000000.  , 10000000.  , 10000000.  , 10000000.  , 10000000.  ,
       10000000.  , 10000000.  , 10000000.  , 10000000.  , 10000000.  ,
       10000000.  ,  7556905.  ,  3240295.  ,  4409304.  ,  5388912.  ,
        1414190.  ,   444847.  ,   550036.7 ,   452219.5 ,   243545.1 ,
         163264.2 ,   128850.9 ,   108392.1 ,    96868.65,    90154.39,
          83498.67,    76720.53,    70937.79,    66821.03,    63916.47,
          61597.41,    60417.97,    60359.64,    60430.77,    59696.93,
          58222.95,    56637.11,    54844.45,    52735.8 ,    50450.12,
          47895.  ,    45134.95,    42075.53,    38557.91,    34843.47,
          31537.89,    29179.72,    28016.06,    27844.87,    28377.06,
          29532.22,    31360.65,    34174.61,    38452.69,    44777.3 ,
          53238.53,    61797.74,    66939.84,    70867.57,    94733.63])

In [ ]:
def preprocessing_v2_rh_mc(data, input_path, target_path):
    npy_input = np.load(input_path)
    npy_target = np.load(target_path)
    
    surface_pressure = npy_input[:, data.ps_index] * \
                        (input_max['state_ps'].values - input_min['state_ps'].values) + \
                        input_mean['state_ps'].values
    
    hyam_component = (data.hyam * data.p0)[np.newaxis,:]
    hybm_component = data.hybm[np.newaxis,:] * surface_pressure[:, np.newaxis]
    
    pressures = hyam_component + hybm_component
    pressures = pressures.reshape(-1,384,60)
    
    pressures_binned = data.zonal_bin_weight_3d(pressures)
    
    npy_input[:,120:180] = 1 - np.exp(-npy_input[:,120:180] * lbd_qn)
    npy_input = (npy_input - input_sub)/input_div
    npy_input = np.where(np.isnan(npy_input), 0, npy_input)
    npy_input = np.where(np.isinf(npy_input), 0, npy_input)
    npy_input[:,120:120+15] = 0
    npy_input[:,60:120] = np.clip(npy_input[:,60:120], 0, 1.2)
    torch_input = torch.tensor(npy_input).float()
    reshaped_target = npy_target.reshape(-1, data.num_latlon, data.target_feature_len)
    return torch_input, reshaped_target, pressures_binned

torch_input, reshaped_target, pressures_binned = preprocessing_v2_rh_mc(data, input_path, target_path)

assert torch_input.shape[0] == reshaped_target.shape[0] * reshaped_target.shape[1]
assert torch_input.shape[0] % data.num_latlon == 0


In [ ]:
# model inference
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for model_name in model_paths.keys():
    model = torch.jit.load(model_paths[model_name]).to(device)
    model.eval()
    model_batch_pred_list = []
    batch_size = data.num_latlon
    with torch.no_grad():
        for i in tqdm(range(0, torch_input.shape[0], batch_size)):
            model_batch_pred = model(torch_input[i : i + batch_size, :]) # inference on batch
            model_batch_pred_list.append(model_batch_pred.cpu().numpy() / out_scale)
            if i > 3840: # this is temporary, remove in final code
                break # this is temporary, remove in final code
    model_preds[model_name] = np.stack(model_batch_pred_list, axis = 0) # 0 axis corresponds to time
    np.save(f'{model_name}_preds.npy', model_preds[model_name])
    
    del model
    del model_batch_pred_list
    gc.collect()

reshaped_target = reshaped_target[:12,...] # this is temporary, remove this later
pressures_binned = pressures_binned[:12,...] # this is temporary, remove this later



In [ ]:
def show_r2(target, preds):
    assert target.shape == preds.shape
    new_shape = (np.prod(target.shape[:-1]), target.shape[-1])
    target_flattened = target.reshape(new_shape)
    preds_flattened = preds.reshape(new_shape)
    r2_scores = np.array([r2_score(target_flattened[:, i], preds_flattened[:, i]) for i in range(308)])
    r2_scores_capped = r2_scores.copy()
    r2_scores_capped[r2_scores_capped < 0] = 0
    return r2_scores, r2_scores_capped

In [ ]:
plt.figure(figsize=(10, 6))
for model_name in model_paths.keys():
    r2_scores[model_name], r2_scores_capped[model_name] = show_r2(reshaped_target, model_preds[model_name])
    label_text = f'{model_labels[model_name]}: {np.mean(r2_scores[model_name]):.3g}' 
    plt.plot(np.arange(data.target_feature_len), r2_scores[model_name], color = model_colors[model_name], label=f"{label_text}")
    
plt.legend()
plt.xlabel('Feature Index')
plt.ylabel(r'$R^2$ Score')
plt.title(r'$R^2$ Score Comparison')

feature_indices = [0, 60, 120, 180, 240, 300, 308]
feature_labels = ['dT', 'dQv', 'dQn (liq+ice)', 'dU', 'dV', 'scalars', '']
plt.xticks(ticks=feature_indices, labels=feature_labels)
plt.ylim(0, 1)
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.savefig(preds_path + 'r2_lines.png')
plt.clf()

In [ ]:

def get_coeff(target, pred):
    rss = np.sum((pred - target)**2, axis = 0)
    tss = np.sum((target - np.mean(target, axis = 0)[None,:,:])**2, axis = 0)
    coeff = 1 - rss/tss
    mask = tss == 0
    coeff[mask] = 1.0 * (rss[mask] == 0) 
    return coeff


In [ ]:

for model_name in model_paths.keys():
    zonal_heating_r2[model_name] = data.zonal_bin_weight_3d(get_coeff(reshaped_target[:,:,:60], \
                                                                      model_preds[model_name][:,:,:60]))[0]
    zonal_moistening_r2[model_name] = data.zonal_bin_weight_3d(get_coeff(reshaped_target[:,:,60:120], \
  

In [ ]:
                                                                        model_preds[model_name][:,:,60:120]))[0]
fig, ax = plt.subplots(2, num_models, figsize = (num_models*12, 18))
y = np.arange(60)
X, Y = np.meshgrid(np.sin(np.pi*lat_bin_mids/180), y)
for i, model_name in enumerate(model_paths.keys()):
    contour_plot_heating = ax[0,i].pcolor(X, Y, zonal_heating_r2[model_name].T, cmap='Blues', vmin = 0, vmax = 1)
    ax[0,i].contour(X, Y, zonal_heating_r2[model_name].T, [0.7], colors='orange', linewidths=[4])
    ax[0,i].contour(X, Y, zonal_heating_r2[model_name].T, [0.9], colors='yellow', linewidths=[4])
    ax[0,i].set_ylim(ax[0,i].get_ylim()[::-1])
    ax[0,i].set_title(f'{model_name} - heating')
    contour_plot = ax[1,i].pcolor(X, Y, zonal_moistening_r2[model_name].T, cmap='Blues', vmin = 0, vmax = 1) # pcolormesh
    ax[1,i].contour(X, Y, zonal_moistening_r2[model_name].T, [0.7], colors='orange', linewidths=[4])
    ax[1,i].contour(X, Y, zonal_moistening_r2[model_name].T, [0.9], colors='yellow', linewidths=[4])
    ax[1,i].set_ylim(ax[1,i].get_ylim()[::-1])
    ax[1,i].set_title(f'{model_name} - moistening')
    ax[1,i].xaxis.set_ticks([np.sin(-50/180*np.pi), 0, np.sin(50/180*np.pi)])
    ax[1,i].xaxis.set_ticklabels(['50$^\circ$S', '0$^\circ$', '50$^\circ$N'])
    ax[1,i].xaxis.set_tick_params(width = 2)
    if i != 0:
        ax[0,i].set_yticks([])
        ax[1,i].set_yticks([])
ax[0,0].set_ylabel("Pressure [hPa]")
ax[0,0].yaxis.set_label_coords(-0.2,-0.09) # (-1.38,-0.09)
ax[0,0].yaxis.set_ticks([1000,800,600,400,200,0])
ax[1,0].yaxis.set_ticks([1000,800,600,400,200,0])
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.12, 0.02, 0.76])
cb = fig.colorbar(contour_plot, cax=cbar_ax)
cb.set_label("Skill Score "+r'$\left(\mathrm{R^{2}}\right)$',labelpad=50.1)
plt.suptitle("Baseline Models Skill for Vertically Resolved Tendencies", y = 0.97)
plt.subplots_adjust(hspace=0.13)
plt.show()
plt.savefig(preds_path + 'press_lat_r2_models.png', bbox_inches='tight', pad_inches=0.1 , dpi = 300)
plt.clf()